In [2]:
import os, sys
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('out/cancer/prompts_obs200_int3_shuf5_anon_steps_gemini-2.5-flash.csv')


In [5]:
df['raw_response']

0     The task is to infer a directed causal graph f...
1     The task is to infer a directed causal graph f...
2     **Step 1: Analyze Observational Data**\n\nLet'...
3     Step-by-step reasoning:\n\n1.  **Analyze Obser...
4     The task is to infer a directed causal graph f...
5     The goal is to infer a directed acyclic graph ...
6     To infer the directed causal graph, I will fol...
7     The task is to infer a directed causal graph f...
8     The task is to infer a directed causal graph (...
9     To infer the directed causal graph, I will fol...
10    Step-by-step reasoning:\n\n1.  **Analyze Obser...
11    The task is to infer a directed causal graph f...
12    The goal is to infer a directed causal graph f...
13    The goal is to infer a directed causal graph f...
14    The provided data presents significant challen...
15    The task is to infer a directed causal graph f...
16    [ERROR] ServerError: 502 Bad Gateway. {'messag...
17    The task is to infer a directed causal gra

In [40]:
df['prediction']

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
48   NaN
49   NaN
Name: prediction, dtype: float64

In [31]:
df_gemini = pd.read_csv("/home/yuen_chen/ENCO/experiments/out/cancer/prompts_obs200_int3_int-stat1_shuf5_rules_gemini-2.5-flash.csv")

In [33]:
df_gemini['prediction']

0     [[0, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1...
1     [[0, 0, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1...
2     [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1...
3     [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1...
4     [[0, 0, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0...
5     [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0...
6     [[0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1...
7     [[0, 1, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1...
8     [[0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1...
9     [[0, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0...
10    [[0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1...
11    [[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1...
12    [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1...
13    [[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1...
14    [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1...
15    [[0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 0...
16    [[0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0...
17    [[0, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 